<div class="alert alert-warning">
<p><img src='../images/hu.png'> A feladathoz az előkészített <a href="https://github.com/mcree/oe-itseclabs">itseclabs</a> Linux virtuális környezetre van szükség!</p>
<p><img src='../images/en.png'> For this task you'll need the prepared <a href="https://github.com/mcree/oe-itseclabs">itseclabs</a> Linux virtual environment.</p>
</div>

## <img src='../images/hu.png'> SSH használata - jelszavas azonosítás

Ez a szakasz az egyszerű, jelszó alapú SSH használatot szemlélteti.

<div class="alert alert-warning">
<p>A demonstráció során az <a href=/lab/workspaces/ssh-instance?clone=ssh target=_self>SSH kliens munkafüzet</a> párhuzamos használatára célszerű!</p>
</div>

## <img src='../images/en.png'> SSH with password based authentication

This section demonstrates simple password based SSH access.

<div class="alert alert-warning">
<p>For demonstration purposes a parallel view of the <a href=/lab/workspaces/ssh-instance?clone=ssh target=_self>SSH client workbook</a> is necessary!</p>
</div>

In [ ]:
# clear terminal screen
ssh-client-cmd "clear"

In [ ]:
# verify that server is reachable with ICMP PING
ssh-client-cmd "ping -c3 server"

In [ ]:
# retireve and display host keys from SSH server
ssh-client-cmd "ssh-keyscan server"

In [ ]:
# save host keys to list of known hosts
ssh-client-cmd "ssh-keyscan server > .ssh/known_hosts"

In [ ]:
# initiate remote connection to the server
ssh-client-cmd "ssh hallgato@server"
# wait for password prompt
sleep 1
# enter user password when prompted
ssh-client-cmd "hallgato"

In [ ]:
# logout from remote server
ssh-client-cmd "exit"

## <img src='../images/hu.png'> SSH használata - kulcs alapú azonosítás

Célja a felhasználói jelszó kliens oldali tárolása / védelme, 
valamint az egyszeri bejelentkezési (SSO - single sign on) lehetőség 
megteremtése. Ugyanaz a felhasználói kulcs számos kiszolgálón 
biztonsággal használható.

## <img src='../images/en.png'> SSH key based authentication

The aim of key based authentication is to store / protect the user
password on the client side, enabling single sign on (SSO) at the same time.
The same user key can be employed in many servers in a secure manner.

In [ ]:
# clear terminal screen
ssh-client-cmd "clear"

In [ ]:
# generate new keypair
ssh-client-cmd "ssh-keygen -t rsa -f mykey"
# wait for password prompt
sleep 1
# private key password will be "secret"
ssh-client-cmd "secret"
# wait for password prompt again
sleep 1
# repeat key password
ssh-client-cmd "secret"

In [ ]:
# check contents of the public key
ssh-client-cmd "cat mykey.pub"

In [ ]:
# copy our *public* key to the server
ssh-client-cmd "ssh-copy-id -i mykey.pub hallgato@server"
# wait for password prompt
sleep 1
# enter user password when prompted
ssh-client-cmd "hallgato"

In [ ]:
# check key based authentication
ssh-client-cmd "ssh -o PasswordAuthentication=no -i mykey hallgato@server"
# wait for the password prompt
sleep 1
# enter password for the *secret key*
ssh-client-cmd "secret"

In [ ]:
# verify public key on the server side
ssh-client-cmd "cat .ssh/authorized_keys"

In [ ]:
# logout from remote server
ssh-client-cmd "exit"

In [ ]:
# start ssh agent process and set up user environment
ssh-client-cmd "eval \$(ssh-agent)"

In [ ]:
# add keypair to the ssh agent
ssh-client-cmd "ssh-add mykey"
# wait for the password prompt
sleep 1
# enter password for the *secret key*
ssh-client-cmd "secret"

In [ ]:
# list active ssh keypairs registered by the agent
ssh-client-cmd "ssh-add -l"

In [ ]:
# try passwordless remote connection
ssh-client-cmd "ssh hallgato@server"

In [ ]:
# logout from remote server
ssh-client-cmd "exit"

## <img src='../images/hu.png'> Házi feladat
Az [SSH kiszolgáló konfiguráció](../docker/ssh-client-server/server/ssh/sshd_config) szerkesztése:

* A "root" felhasználó bejelentkezésének tiltása
* Jelszó alapú bejelentkezés tiltása (kulcs alapú azonosítás kényszerítése)
* 128 bites AES titkosítás kikapcsolása

További információk: https://man.openbsd.org/sshd_config

## <img src='../images/en.png'> Homework
Modify the [SSH Server Configuration](../docker/ssh-client-server/server/ssh/sshd_config) in order to:

* Disable all forms of login of user "root"
* Disable password based login for all users (enforce key based authentication)
* Disable 128 bit AES encryption profile

For more information see: https://man.openbsd.org/sshd_config

In [ ]:
# restart SSH server with new configuration
cd /home/hallgato/docker/ssh-client-server/
docker-compose restart server

In [ ]:
# verify server logs - check for configuration file errors
cd /home/hallgato/docker/ssh-client-server/
docker-compose logs --tail=25 server